In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [2]:
%pip install -q mctx clu pynvml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.0/101.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 87.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
dopamine-r

In [3]:
# Example for CUDA 12
!pip install -q -U "jax[cuda12]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.1/123.1 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.2/644.2 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 MB 12.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.

In [ ]:
import jax
import jax.numpy as jnp
from jax import lax, random, jit, vmap, pmap

import flax.linen as nn
from flax.training import train_state
from flax import serialization # Model serialization
from flax import jax_utils # Multi-device utils

import optax
import mctx

from functools import partial
from typing import Tuple, Dict
import chex
import time

import numpy as np
import os

from clu import metric_writers
from pynvml import *

nvmlInit()
handle0 = nvmlDeviceGetHandleByIndex(0)
handle1 = nvmlDeviceGetHandleByIndex(1)


In [ ]:
class AlphaZeroNet(nn.Module):
    action_size: int

    @nn.compact
    def __call__(self, x):
        x = nn.Dense(features=256)(x)
        x = nn.relu(x)
        x = nn.Dense(features=256)(x)
        x = nn.relu(x)
        policy_logits = nn.Dense(features=self.action_size)(x)
        value_logit = nn.Dense(features=1)(x)
        value = nn.tanh(value_logit)
        return policy_logits, jnp.squeeze(value, axis=-1)

class TrainState(train_state.TrainState):
    pass

def create_train_state(rng, learning_rate, action_size, obs_shape):
    model = AlphaZeroNet(action_size=action_size)
    params = model.init(rng, jnp.ones(obs_shape))['params']
    tx = optax.adam(learning_rate)
    return TrainState.create(apply_fn=model.apply, params=params, tx=tx)

@partial(jit, static_argnums=(2,))
def get_model_outputs(state: TrainState, obs: chex.Array, action_size: int):
    policy_logits, value = state.apply_fn({'params': state.params}, obs)
    return policy_logits, value

In [ ]:
def run_self_play_and_train(
    key: chex.PRNGKey,
    train_state: TrainState,
    env: AaduPuliAattamJAX,
    num_simulations: int,
    batch_size: int,
    game_length: int
):
    
    @jit
    def single_game_step(state_and_key, _):
        state, key = state_and_key
        current_obs = env.get_observation(state)
        policy_logits, value = get_model_outputs(train_state, current_obs, env.total_actions)
        legal_mask = env.legal_action_mask(state)

        batched_logits = jnp.expand_dims(policy_logits, axis=0)
        batched_value = jnp.expand_dims(value, axis=0)
        
        batched_embedding_state = jax.tree.map(lambda x: jnp.expand_dims(x, axis=0), state)

        root = mctx.RootFnOutput(
            prior_logits=jnp.where(jnp.expand_dims(legal_mask, axis=0), batched_logits, -jnp.inf),
            value=batched_value,
            embedding=batched_embedding_state
        )

        def recurrent_fn(params, rng_key, action, embedding_state):
            unbatched_state = jax.tree.map(lambda x: jnp.squeeze(x, axis=0), embedding_state)
            
            next_state, reward = env.step(unbatched_state, action[0])
            next_obs = env.get_observation(next_state)
            logits, val = get_model_outputs(train_state, next_obs, env.total_actions)
            discount = jnp.array(1.0, dtype=jnp.float32) * (1 - next_state.terminated)

            recurrent_fn_output = mctx.RecurrentFnOutput(
                reward=jnp.expand_dims(reward, axis=0),
                discount=jnp.expand_dims(discount, axis=0),
                prior_logits=jnp.where(jnp.expand_dims(env.legal_action_mask(next_state), axis=0), jnp.expand_dims(logits, axis=0), -jnp.inf),
                value=jnp.expand_dims(val, axis=0)
            )
            batched_next_state = jax.tree.map(lambda x: jnp.expand_dims(x, axis=0), next_state)

            return recurrent_fn_output, batched_next_state
            
        policy_output = mctx.muzero_policy(
            params=train_state.params,
            rng_key=key,
            root=root,
            recurrent_fn=recurrent_fn,
            num_simulations=num_simulations,
        )
        
        action = jnp.squeeze(policy_output.action, axis=0)
        action_weights = jnp.squeeze(policy_output.action_weights, axis=0)
        
        next_state, _ = env.step(state, action)
        search_pi = jax.nn.softmax(action_weights)
        mcts_value = jnp.squeeze(policy_output.search_tree.summary().value, axis=0)
        
        data = {'obs': current_obs, 'pi': search_pi, 'value': mcts_value}
        key, _ = random.split(key)
        return (next_state, key), data

    keys = random.split(key, batch_size)
    # Parallelize the environment reset across the batch using vmap
    # This creates batch_size independent game instances, each starting from the initial state
    # vmap applies the reset function vectorized over the batch dimension
    vmapped_initial_state = vmap(lambda _: env.reset())(jnp.arange(batch_size))

    # Run the self-play loop in parallel across the batch
    # lax.scan iterates over game_length steps, with vmap applying single_game_step to each game in the batch
    # This parallelizes the game stepping across all batch_size games simultaneously
    (_, _), trajectory_data = lax.scan(vmap(single_game_step), (vmapped_initial_state, keys), None, length=game_length)
    
    obs_batch = jnp.reshape(trajectory_data['obs'], (-1, *trajectory_data['obs'].shape[2:]))
    pi_batch = jnp.reshape(trajectory_data['pi'], (-1, *trajectory_data['pi'].shape[2:]))
    value_batch = jnp.reshape(trajectory_data['value'], (-1,))

    @jit
    def train_step(state, batch):
        obs, target_pi, target_value = batch
        def loss_fn(params):
            policy_logits, pred_value = state.apply_fn({'params': params}, obs)
            value_loss = jnp.mean((pred_value - target_value) ** 2)
            policy_loss = -jnp.mean(jnp.sum(target_pi * jax.nn.log_softmax(policy_logits), axis=-1))
            total_loss = value_loss + policy_loss
            return total_loss, (value_loss, policy_loss)
            
        (loss, (v_loss, p_loss)), grads = jax.value_and_grad(loss_fn, has_aux=True)(state.params)

        # Average gradients across all devices to synchronize the model updates
        # lax.pmean computes the mean of gradients across the 'devices' axis
        # This ensures all devices contribute equally to the parameter updates in data-parallel training
        grads = lax.pmean(grads, axis_name='devices')
        new_state = state.apply_gradients(grads=grads)
        metrics = {'total_loss': loss, 'value_loss': v_loss, 'policy_loss': p_loss}
        return new_state, metrics

    new_train_state, metrics = train_step(train_state, (obs_batch, pi_batch, value_batch))
    return new_train_state, metrics

def save_checkpoint(train_state, path):
  os.makedirs(os.path.dirname(path), exist_ok=True)
  bytes_output = serialization.to_bytes(train_state)
  with open(path, "wb") as f:
    f.write(bytes_output)
  # print(f"Checkpoint saved to {path}")

def load_checkpoint(train_state_shape, path):
    with open(path, "rb") as f:
        bytes_input = f.read()
    return serialization.from_bytes(train_state_shape, bytes_input)

log_dir = '/kaggle/working/logs'
writer = metric_writers.create_default_writer(log_dir)

In [ ]:
if __name__ == '__main__':
    # 1. jax_utils.replicate-> Distributes model parameters across devices
    # 2. pmap-> Runs the training function in parallel across devices
    # 3. Each device processes its own batch of games independently
    # 4. Gradients are averaged across devices using lax.pmean
    # 5. jax_utils.unreplicate-> Collects metrics from all devices
    # 6. jax.device_get-> Transfers state from device to host for saving
    NUM_DEVICES = len(jax.devices())
    BATCH_SIZE = 512 # Per device
    GAME_LENGTH_PER_ITER = 10
    NUM_SIMULATIONS = 32
    LEARNING_RATE = 1e-3
    TRAINING_STEPS = 5000
    STEP_UPDATE = 100
    print("--- AlphaZero on Aadu Puli Aattam ---")
    print(f"Detected {NUM_DEVICES} JAX devices: {[d.platform for d in jax.devices()]}")
    if NUM_DEVICES == 0 or 'gpu' not in jax.devices()[0].platform.lower():
        print("WARNING: No GPU detected. Training will be very slow.")
    
    env = AaduPuliAattamJAX()
    key = random.PRNGKey(42)
    obs_shape = env.get_observation(env.reset()).shape
    initial_train_state = create_train_state(key, LEARNING_RATE, env.total_actions, obs_shape)

    # Replicate the training state across all available devices (GPUs)
    # This creates a copy of the model parameters on each device for parallel training
    # jax_utils.replicate distributes the single-device state to all devices
    replicated_train_state = jax_utils.replicate(initial_train_state)

    # pmap creates a parallel version of the training function
    # pmap applies the function across devices in parallel, with axis_name='devices' for cross-device communication
    # This enables data parallelism, that is, each device runs the self-play and training on its own batch
    pmapped_main_fn = pmap(
        partial(
            run_self_play_and_train,
            env=env,
            num_simulations=NUM_SIMULATIONS,
            batch_size=BATCH_SIZE,
            game_length=GAME_LENGTH_PER_ITER
        ),
        axis_name='devices'
    )

    print("\nStarting training...")
    total_batch_size = BATCH_SIZE * max(1, NUM_DEVICES)
    print(f"Total batch size across all devices: {total_batch_size}")
    print(f"Data generated per iteration: {total_batch_size * GAME_LENGTH_PER_ITER} state-action pairs")
    
    for step in range(TRAINING_STEPS):
        start_time = time.time()
        
        step_keys = random.split(key, max(1, NUM_DEVICES))
        key, _ = random.split(key)
        
        # Execute the parallel training function across all devices
        # Each device runs self-play and training on its own batch of games
        # The function returns updated states and metrics from each device
        replicated_train_state, metrics = pmapped_main_fn(step_keys, replicated_train_state)
        
        # Unreplicate the metrics from device-parallel format back to single-device format
        # This gathers the metrics from all devices and combines them (e.g., averaging losses)
        # jax_utils.unreplicate converts from device-distributed to host format
        metrics = jax_utils.unreplicate(metrics)
        
        end_time = time.time()
        
        if step % STEP_UPDATE == 0:
            print(
                f"Step: {step}, "
                f"Time/iter: {end_time - start_time:.2f}s, "
                f"Total Loss: {metrics['total_loss']:.4f}, "
                f"Value Loss: {metrics['value_loss']:.4f}, "
                f"Policy Loss: {metrics['policy_loss']:.4f}"
            )
    
            # GPU stats for both GPUs
            utilization0 = nvmlDeviceGetUtilizationRates(handle0)
            memory0 = nvmlDeviceGetMemoryInfo(handle0)
            utilization1 = nvmlDeviceGetUtilizationRates(handle1)
            memory1 = nvmlDeviceGetMemoryInfo(handle1)
    
            # Logging into TensorBoard
            writer.write_scalars(step, {
                'Loss/Total': metrics['total_loss'],
                'Loss/Value': metrics['value_loss'],
                'Loss/Policy': metrics['policy_loss'],
                'GPU/GPU_0_Utilization_Percent': utilization0.gpu,
                'GPU/GPU_0_Memory_Used_MiB': memory0.used / 1024**2,
                'GPU/GPU_1_Utilization_Percent': utilization1.gpu,
                'GPU/GPU_1_Memory_Used_MiB': memory1.used / 1024**2
            })
            writer.flush()

            # de-parallelizes the state so it can be serialized and saved to disk
            state_to_save = jax.device_get( # transfers the data from device memory to CPU memory
                jax.tree_util.tree_map(lambda x: x[0], # selects the state from device 0
                replicated_train_state))
            save_checkpoint(state_to_save, f'/kaggle/working/apa_jax_e2e.ckpt')

    nvmlShutdown()

2025-09-15 07:30:57.991534: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757921458.352387      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757921458.459806      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1757921478.019183      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1757921478.020031      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability:

--- AlphaZero on Aadu Puli Aattam ---
Detected 2 JAX devices: ['gpu', 'gpu']

Starting training...
Total batch size across all devices: 1024
Data generated per iteration: 10240 state-action pairs
Step: 0, Time/iter: 18.34s, Total Loss: 170.8706, Value Loss: 165.7854, Policy Loss: 5.0852
Step: 100, Time/iter: 3.20s, Total Loss: 76.7580, Value Loss: 71.7142, Policy Loss: 5.0438
Step: 200, Time/iter: 2.89s, Total Loss: 54.0544, Value Loss: 49.0128, Policy Loss: 5.0416
Step: 300, Time/iter: 2.25s, Total Loss: 42.8682, Value Loss: 37.8275, Policy Loss: 5.0407
Step: 400, Time/iter: 2.10s, Total Loss: 35.6216, Value Loss: 30.5811, Policy Loss: 5.0405
Step: 500, Time/iter: 2.03s, Total Loss: 33.3305, Value Loss: 28.2901, Policy Loss: 5.0404
Step: 600, Time/iter: 2.26s, Total Loss: 31.8519, Value Loss: 26.8114, Policy Loss: 5.0404
Step: 700, Time/iter: 1.94s, Total Loss: 30.3678, Value Loss: 25.3274, Policy Loss: 5.0404
Step: 800, Time/iter: 2.00s, Total Loss: 30.1098, Value Loss: 25.0695, Poli